In [ ]:
#import libraries
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

In [ ]:
# bring in csv files
food_security_hh = "data/data-EFK41.csv"
obesity_df = "data/data-xeuft.csv"
grand_food_security_df = "data/XVKxd.csv"

